<img src="Images/cover_readme.png" alt="Example Image" style="width:1400px; height:400px;" >

--- 
# **DeepCellMap Laborarory - Find best cell segmentation adapted to each dataset and images**
---

Depending on the dataset and the type of data. Cell segmentation can be done using different steps. In this notebook, differents steps are tested on a bunch of tiles pre-selectionned. Results are saved and visualised thanks to an html file. 

In [2]:
from grabber import *  # This imports your common setup code from grabber.py
%precision 2
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import os
# from pylibCZIrw import czi as pycziDeep
from utils.util_colors_drawing import *
from utils.util_fig_display import *
from utils.util import *
from preprocessing import filter, slide, tiles
from stat_analysis import deep_cell_map
from segmentation_classification import segmentation
from segmentation_classification.classification import ModelClassification, segment_classify_cells_wsi

from config.datasets_config import *

#temporaire 
from config.html_generation_config import HtmlGenerationConfig
from config.dataset_management import take_config_from_dataset
%matplotlib inline 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset selection 

In [3]:
display(dataframe_dropdown)
dataset_name = dataframe_dropdown.value

Dropdown(description='Select dataset:', options={'generated_rois': 'generated_rois', 'ihc_microglia_fetal_huma…

In [9]:
dataset_name = dataframe_dropdown.value
dataset_config = take_config_from_dataset(dataset_name)
print(blue("We are playing with the dataset: {}".format(dataset_config.dataset_name)))


We are playing with the dataset: covid_data_immunofluorescence


# 1. Possible steps to segment cells

In [5]:
param_best_cellpose =dataset_config.param_best_cellpose
step_possibilities = dict({
            "cellpose" : param_best_cellpose,

            #Binarization steps 
            "multi_otsu_thresholding" :  ["dont_consider_background", 40],
            "otsu_thresholding" : None,
            "manual_threshold" : 10,
            "rgb_threshold_based_binarization_microglia_IHC": None,

            #Morphological operations 
            "erosion": 2,
            "dilation" : 4,
            "dilation2":3,
            "opening":5,

            #Other operations 
            "fill_holes" : None,
            "filter_center_cells" : dataset_config.tile_width,
            "remove_small_objects" : 150,
            "remove_large_objects" : 10000,
            "rgb_to_eosin" : None,
        })

# 2. Definition of the different steps by channel

The different cell segmentation steps can be common to all the images but can also be specified for each image.  

Once a good sequence of steps works for many images, the dictionnary of the different steps can be given as value to the key *default* of the parameter **dataset_config.cell_segmentation_param_by_cannnel** 

**If this shared sequence of steps doesn't work for an image N**, a new sequence can be added to the dictionnary **cell_segmentation_param_by_cannnel** specifying the key as *image_N*

*This is shared with the tissue segmentation method* 

In [10]:
#Define different steps of cell segmentation for each channel
cell_segmentation_param = dict({
    # 0: {
    #     "multi_otsu_thresholding" :  ["dont_consider_background", 80],
    #     "erosion" : 1,
    #     "dilation" : 3,
    #     "fill_holes" : None,
    #     "remove_small_objects" : 200,
    #     "remove_large_objects" : 20000,
    #     "dilation2":2,
    # },
    1: {#R
        # "test_cas1__cas_2" :  {"dilation_test":2, "thresh_fraction_cells_discovered":0.15,"multi_otsu":False, "param_erosion_cas_1":3, "param_dil_cas_2":0,"min_thresh_second_binarization":80,"dont_consider_background":True},
        # "remove_small_objects" : 250,
        # "remove_large_objects" : 10000,
        "multi_otsu_thresholding" :  [None, 25],
        "opening":1,
        "dilation" : 2,
        "remove_small_objects" : 200,
        "remove_large_objects" : 10000,
        },
    2: {#G
        "multi_otsu_thresholding" : [None,-1],
        "remove_small_objects" : 250,
        "remove_large_objects" : 10000,
        },
    3: {#B
        "cellpose" : param_best_cellpose,

        }
    })
# "min_thresh_second_binarization":10

# cell_segmentation_param_by_cannnel = dataset_config.cell_segmentation_param_by_cannnel

In [7]:
#IHC 

cell_segmentation_param = dict(
    {
        "rgb_to_d" : "mean_channels",
        "multi_otsu_thresholding" : [None,-1],
        "opening" : 2,
        "dilation" : 2,
        "fill_holes" : None,
        "remove_small_objects": 400,
    }
)

# 3.Apply segmentation to different tiles 

In [ ]:
from_image, to_image = 0,0
#OR 
image_list = [1]

n_tiles_per_channel = 2
channels_to_segment = [3]

#Define segmentor
cell_segmentor = segmentation.ModelSegmentation(dataset_config=dataset_config, object_to_segment = "cells", cell_segmentation_param=cell_segmentation_param)
#Find tiles to test cell segmentation
tiles_list, tiles_names  = tiles.get_tiles_to_segment(dataset_config, from_image, to_image, image_list=image_list, n_tiles_per_channel = n_tiles_per_channel,channels_to_segment = channels_to_segment)
#Perform segmentation
list_images_processed, list_process_names = cell_segmentor.test_cell_segmentation_from_tiles(tiles_list,tiles_names,channels_to_segment = channels_to_segment, save_html = True)
#Save different steps of cell segmentation
segmentation.ModelSegmentation.laboratory_segmentation_save_images(dataset_config,list_images_processed, list_process_names, segmentation_type = "cells",channels_to_segment=channels_to_segment,image_list=image_list)

# Just results of the different configurations 

In [41]:

n_tiles_per_channel = 5
image_list = [2]
channels_to_segment = [1]

tiles_list, tiles_names  = tiles.get_tiles_to_segment(dataset_config, from_image, to_image, image_list=image_list, n_tiles_per_channel = n_tiles_per_channel)

segmentation.test_existing_segmentation_config(dataset_config,tiles_list, tiles_names,channels_to_segment,image_list=image_list)



HTML saved at /export/home1/users/mcbs/perochon/Documents/Projet_Microglial_cells/code/projet_ens/output/cancer_data_immunofluorescence/1_image_pre_processing/cells_filtering_laboratory/img_01/existing_config_channel_1.html


# Test existing configurations to the new image 

In [12]:
from_image, to_image = 0,0
image_list = [2]
n_tiles_per_channel = 5
channels_to_segment = [1]


tiles_list, tiles_names  = tiles.get_tiles_to_segment(dataset_config, from_image, to_image, image_list=image_list, n_tiles_per_channel = n_tiles_per_channel)


image_list = [9]
n_tiles_per_channel = 5
channels_to_segment = [1]
segmentation.test_existing_segmentation_config_with_details(dataset_config,tiles_list, tiles_names,channels_to_segment)



default
dict_processing_steps {0: {'multi_otsu_thresholding': ['dont_consider_background', 80], 'erosion': 1, 'dilation': 3, 'fill_holes': None, 'filter_center_cells': 1024, 'remove_small_objects': 200, 'remove_large_objects': 20000, 'dilation2': 2}, 1: {'test_cas1__cas_2': {'dilation_test': 2, 'thresh_fraction_cells_discovered': 0.15, 'multi_otsu': False, 'param_erosion_cas_1': 3, 'param_dil_cas_2': 0, 'min_thresh_second_binarization': 80, 'dont_consider_background': True}, 'filter_center_cells': 1024, 'remove_small_objects': 250, 'remove_large_objects': 10000}, 2: {'test_cas1__cas_2': {'dilation_test': 2, 'thresh_fraction_cells_discovered': 0.15, 'multi_otsu': False, 'param_erosion_cas_1': 3, 'param_dil_cas_2': 0, 'min_thresh_second_binarization': 80, 'dont_consider_background': True}, 'fill_holes': None, 'remove_small_objects': 200, 'remove_large_objects': 10000, 'filter_center_cells': 1024}, 3: {'otsu_thresholding': [None, 70], 'fill_holes': None, 'remove_small_objects': 200, 'remo